In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)

os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns

import pickle 
import logging


from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/plaid_credit
/Users/shashankgupta/Documents/code/git_project/plaid_credit/Code


In [2]:
querySno = SQLQuery('snowflake')
base = querySno(r""" 
with default_flags as (
select distinct lending_business_id, business_id,loan_month,
max(case when emi_no = 1 then due_date end) over(partition by lending_business_id) as first_duedate, 
max(case when emi_no = 2 then due_date end) over(partition by lending_business_id) as second_duedate, 
max(case when emi_no = 3 then due_date end) over(partition by lending_business_id) as third_duedate,
max(case when emi_no = 4 then due_date end) over(partition by lending_business_id) as fourth_duedate,
max(case when emi_no = 5 then due_date end) over(partition by lending_business_id) as fifth_duedate,
max(case when emi_no = 6 then due_date end) over(partition by lending_business_id) as sixth_duedate,
max(case when emi_no = 1 then defalt end) over(partition by lending_business_id) as fpd, 
max(case when emi_no = 2 then defalt end) over(partition by lending_business_id) as spd, 
max(case when emi_no = 3 then defalt end) over(partition by lending_business_id) as tpd,
max(case when emi_no = 4 then defalt end) over(partition by lending_business_id) as frpd,
max(case when emi_no = 5 then defalt end) over(partition by lending_business_id) as ffpd,
max(case when emi_no = 6 then defalt end) over(partition by lending_business_id) as sxpd,
max(case when emi_no = 1 then total_emi end) over(partition by lending_business_id) as first_emi_amount,
max(case when emi_no = 2 then total_emi end) over(partition by lending_business_id) as second_emi_amount,
max(case when emi_no = 3 then total_emi end) over(partition by lending_business_id) as third_emi_amount,
max(case when emi_no = 4 then total_emi end) over(partition by lending_business_id) as fourth_emi_amount,
max(case when emi_no = 5 then total_emi end) over(partition by lending_business_id) as fifth_emi_amount,
max(case when emi_no = 6 then total_emi end) over(partition by lending_business_id) as sixth_emi_amount
from (
select a.*,json_extract_path_text(c.predict_meta, 'business_id') as b1_id, json_extract_path_text(c.predict_meta, 'data.business_id') as b2_id,
case when b1_id is null then b2_id else b1_id end as business_id,
month(a.created_at) as month_no,
case when paid_on_time = 'true' then 0
when paid_on_time = 'false' then 1 else null end as default_flag,
dense_rank() over(partition by a.lending_business_id order by due_date) as emi_no,
min(a.created_at) over (partition by a.lending_business_id) as min_created_at,
month(min_created_at) as loan_month,
max(default_flag) over(partition by a.lending_business_id, due_date) as defalt,
sum(due_amount) over(partition by a.lending_business_id, due_date) as amount_due,
sum(payment) over(partition by a.lending_business_id, due_date) as payment_done,
amount_due/100 + payment_done/100 as total_emi
from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES" a
join FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS c on a.lending_business_id = c.lending_business_id
where date(due_date) <= date('2023-06-05')
-- and a.lending_business_id = '909564ef-6b22-4303-9df5-fc4e892fef01'
)
)
, first_cure as (
select *,transaction_date as first_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) rnk
from (
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as first_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.second_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where first_amount_cured >= first_emi_amount
)
where rnk = 1
)
, second_cure as (
select *, transaction_date as second_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
t.running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as second_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.second_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where second_amount_cured >= first_emi_amount + second_emi_amount
)
where rnk = 1
)

,third_cure as (
select *, transaction_date as third_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount, t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as third_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where third_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount
)
where rnk = 1
)

,fourth_cure as (
select *, transaction_date as fourth_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount,b.fourth_emi_amount ,t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,b.fourth_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as fourth_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where fourth_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount + fourth_emi_amount
)
where rnk = 1
)

,fifth_cure as (
select *, transaction_date as fifth_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount,b.fourth_emi_amount,b.fifth_emi_amount ,t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,b.fourth_duedate, b.fifth_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as fifth_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where fifth_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount + fourth_emi_amount + fifth_emi_amount
)
where rnk = 1
)

,sixth_cure as (
select *, transaction_date as sixth_cure_date from (
select *, row_number() over(partition by business_id order by transaction_date) as rnk
from(
select b.lending_business_id, t.business_id, t.transaction_date, b.first_emi_amount,b.second_emi_amount,b.third_emi_amount,b.fourth_emi_amount,
b.fifth_emi_amount,b.sixth_emi_amount, t.description, 
running_balance,b.first_duedate,b.second_duedate,b.third_duedate,b.fourth_duedate, b.fifth_duedate, b.sixth_duedate,
sum(-amount) over(partition by t.business_id order by transaction_date) as sixth_amount_cured
from default_flags b left join prod_db.data.transactions t on t.business_id = b.business_id
-- join first_cure c on b.lending_business_id = c.lending_business_id
where date(transaction_date) >= date(b.first_duedate)
-- and date(transaction_date) < date(b.third_duedate)
and description ILIKE any('Novo Funding%%')
and amount < 0
)
where sixth_amount_cured >= first_emi_amount + second_emi_amount + third_emi_amount + fourth_emi_amount + fifth_emi_amount + sixth_emi_amount
)
where rnk = 1
)

select a.lending_business_id, a.business_id, a.first_duedate,date(f.first_cure_date)as first_cure_date, a.second_duedate,date(s.second_cure_date)as second_cure_date, 
a.third_duedate,date(t.third_cure_date) as third_cure_date, a.fourth_duedate,date(fr.fourth_cure_date) as fourth_cure_date, a.fifth_duedate, date(ff.fifth_cure_date) as fifth_cure_date,
a.sixth_duedate, date(sx.sixth_cure_date) as sixth_cure_date,
fpd, spd, tpd, frpd, ffpd,sxpd,
case when fpd = 1 and first_amount_cured >= a.first_emi_amount then 1 else 0 end as first_cure_flag,
case when spd = 1 and second_amount_cured >= a.first_emi_amount + a.second_emi_amount then 1 else 0 end as second_cure_flag,
case when tpd = 1 and third_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount then 1 else 0 end as third_cure_flag,
case when frpd = 1 and fourth_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount + a.fourth_emi_amount then 1 else 0 end as fourth_cure_flag,
case when ffpd = 1 and fifth_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount + a.fourth_emi_amount + a.fifth_emi_amount then 1 else 0 end as fifth_cure_flag,
case when sxpd = 1 and sixth_amount_cured >= a.first_emi_amount + a.second_emi_amount + a.third_emi_amount + a.fourth_emi_amount + a.fifth_emi_amount + a.sixth_emi_amount then 1 else 0 end as sixth_cure_flag,
case when first_cure_flag = 1 then datediff(day,a.first_duedate,f.first_cure_date) else null end as first_cure_days,
case when second_cure_flag = 1 then datediff(day,a.second_duedate,s.second_cure_date) else null end as second_cure_days,
case when third_cure_flag = 1 then datediff(day,a.third_duedate,t.third_cure_date) else null end as third_cure_days,
case when fourth_cure_flag = 1 then datediff(day,a.fourth_duedate,fr.fourth_cure_date) else null end as fourth_cure_days,
case when fifth_cure_flag = 1 then datediff(day, a.fifth_duedate, ff.fifth_cure_date) else null end as fifth_cure_days,
case when sixth_cure_flag = 1 then datediff(day, a.sixth_duedate, sx.sixth_cure_date) else null end as sixth_cure_days,
a.first_emi_amount, f.first_amount_cured, a.second_emi_amount,(s.second_amount_cured - f.first_amount_cured) as second_amt_cured, a.third_emi_amount, 
(t.third_amount_cured - second_amt_cured) as third_amt_cured, a.fourth_emi_amount, (fr.fourth_amount_cured - third_amt_cured) as fourth_amt_cured,
(ff.fifth_amount_cured - fourth_amt_cured) as fifth_amt_cured, (sx.sixth_amount_cured - fifth_amt_cured) as sixth_amt_cured
from default_flags a 
left join first_cure f on a.lending_business_id = f.lending_business_id
left join second_cure s on a.lending_business_id = s.lending_business_id
left join third_cure t on a.lending_business_id = t.lending_business_id
left join fourth_cure fr on a.lending_business_id = fr.lending_business_id
left join fifth_cure ff on a.lending_business_id = ff.lending_business_id
left join sixth_cure sx on a.lending_business_id = sx.lending_business_id
where a.first_duedate <= date('2023-06-05')
-- and a.business_id = '111eb19b-cd06-4943-b9f5-c111ae38189e'
order by first_duedate desc
""")

In [3]:
base.shape

(2835, 42)

In [4]:
base.head()

,lending_business_id,business_id,first_duedate,first_cure_date,second_duedate,second_cure_date,third_duedate,third_cure_date,fourth_duedate,fourth_cure_date,...,first_emi_amount,first_amount_cured,second_emi_amount,second_amt_cured,third_emi_amount,third_amt_cured,fourth_emi_amount,fourth_amt_cured,fifth_amt_cured,sixth_amt_cured
0,6b3e7079-6cc7-4103-8bb1-2c5c9455e888,34ef815c-eba6-4acb-a6f0-a5c9ece158e6,2023-06-05,2023-06-05,None,None,None,None,None,None,...,933.33,933.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,06296573-9834-4351-b450-c59f44e9741e,c4cec416-702f-4067-ba46-84088fbc0a5f,2023-06-05,None,None,None,None,None,None,None,...,235.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c74679c1-92d2-43ac-8b73-3e3ed8b8000a,136032b3-d2f5-43ab-a9d2-33230517857f,2023-06-05,2023-06-05,None,None,None,None,None,None,...,466.00,466.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,028fa7c5-b2f8-45a2-a738-b670de867b94,32947d50-65c5-448a-9bf1-867ab1a3e52f,2023-06-05,2023-06-05,None,None,None,None,None,None,...,98.33,98.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6e7c058d-8863-4c6c-8547-b3bfa0a022ef,198af8d7-fcf0-4636-800b-2b49cdbfbb2d,2023-06-05,None,None,None,None,None,None,None,...,282.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
base['everDPD'] = base.apply(lambda x: max(x['fpd'],x['spd'],x['tpd'],x['frpd'],x['ffpd'],x['sxpd']),axis=1)

##### everDPD calculations

In [7]:
base['fpd_plus_3'] = 0
base['spd_plus_3'] = 0
base['tpd_plus_3'] = 0
base['frpd_plus_3'] = 0
base['ffpd_plus_3'] = 0
base['sxpd_plus_3'] = 0

base.loc[(base['fpd']==1) & (base['first_cure_days']>=3), 'fpd_plus_3'] = 1
base.loc[(base['spd']==1) & (base['second_cure_days']>=3), 'spd_plus_3'] = 1
base.loc[(base['tpd']==1) & (base['third_cure_days']>=3), 'tpd_plus_3'] = 1
base.loc[(base['frpd']==1) & (base['fourth_cure_days']>=3), 'frpd_plus_3'] = 1
base.loc[(base['ffpd']==1) & (base['fifth_cure_days']>=3), 'ffpd_plus_3'] = 1
base.loc[(base['sxpd']==1) & (base['sixth_cure_days']>=3), 'sxpd_plus_3'] = 1

In [8]:
base['fpd_plus_15'] = 0
base['spd_plus_15'] = 0
base['tpd_plus_15'] = 0
base['frpd_plus_15'] = 0
base['ffpd_plus_15'] = 0
base['sxpd_plus_15'] = 0

base.loc[(base['fpd']==1) & (base['first_cure_days']>=15), 'fpd_plus_15'] = 1
base.loc[(base['spd']==1) & (base['second_cure_days']>=15), 'spd_plus_15'] = 1
base.loc[(base['tpd']==1) & (base['third_cure_days']>=15), 'tpd_plus_15'] = 1
base.loc[(base['frpd']==1) & (base['fourth_cure_days']>=15), 'frpd_plus_15'] = 1
base.loc[(base['ffpd']==1) & (base['fifth_cure_days']>=15), 'ffpd_plus_15'] = 1
base.loc[(base['sxpd']==1) & (base['sixth_cure_days']>=15), 'sxpd_plus_15'] = 1

In [9]:
base['fpd_plus_30'] = 0
base['spd_plus_30'] = 0
base['tpd_plus_30'] = 0
base['frpd_plus_30'] = 0
base['ffpd_plus_30'] = 0
base['sxpd_plus_30'] = 0

base.loc[(base['fpd']==1) & (base['first_cure_days']>=30), 'fpd_plus_30'] = 1
base.loc[(base['spd']==1) & (base['second_cure_days']>=30), 'spd_plus_30'] = 1
base.loc[(base['tpd']==1) & (base['third_cure_days']>=30), 'tpd_plus_30'] = 1
base.loc[(base['frpd']==1) & (base['fourth_cure_days']>=30), 'frpd_plus_30'] = 1
base.loc[(base['ffpd']==1) & (base['fifth_cure_days']>=30), 'ffpd_plus_30'] = 1
base.loc[(base['sxpd']==1) & (base['sixth_cure_days']>=30), 'sxpd_plus_30'] = 1

##### EverDPDs

In [10]:
base['everDPD_3'] = base.apply(lambda x: max(x['fpd_plus_3'],x['spd_plus_3'],x['tpd_plus_3'],x['frpd_plus_3'],x['ffpd_plus_3'],x['sxpd_plus_3']),axis=1)
base['everDPD_15'] = base.apply(lambda x: max(x['fpd_plus_15'],x['spd_plus_15'],x['tpd_plus_15'],x['frpd_plus_15'],x['ffpd_plus_15'],x['sxpd_plus_15']),axis=1)
base['everDPD_30'] = base.apply(lambda x: max(x['fpd_plus_30'],x['spd_plus_30'],x['tpd_plus_30'],x['frpd_plus_30'],x['ffpd_plus_30'],x['sxpd_plus_30']),axis=1)

In [11]:
print("total accounts = {}".format(base.shape[0]))
print("everDPD_count = {}, everDPD% = {}".format(base['everDPD'].sum(),base['everDPD'].sum()/len(base)))
print("everDPD_3_count = {}, everDPD_3% = {}".format(base['everDPD_3'].sum(),base['everDPD_3'].sum()/len(base)))
print("everDPD_15_count = {}, everDPD_15% = {}".format(base['everDPD_15'].sum(),base['everDPD_15'].sum()/len(base)))
print("everDPD_30_count = {}, everDPD_30% = {}".format(base['everDPD_30'].sum(),base['everDPD_30'].sum()/len(base)))

total accounts = 2835
everDPD_count = 928.0, everDPD% = 0.327336860670194
everDPD_3_count = 589, everDPD_3% = 0.20776014109347443
everDPD_15_count = 404, everDPD_15% = 0.14250440917107585
everDPD_30_count = 315, everDPD_30% = 0.1111111111111111


In [13]:
cols = ['lending_business_id','fpd','spd','tpd','frpd','ffpd','sxpd','sixth_cure_days','everDPD','everDPD_15']

In [14]:
# Fully aged accounts (till 5th Jun'23)
base.loc[~base['sxpd'].isnull(),cols].shape

(166, 10)

In [15]:
base.loc[base['sxpd']==1,cols].shape

(73, 10)

In [16]:
base.head()

,lending_business_id,business_id,first_duedate,first_cure_date,second_duedate,second_cure_date,third_duedate,third_cure_date,fourth_duedate,fourth_cure_date,...,sxpd_plus_15,fpd_plus_30,spd_plus_30,tpd_plus_30,frpd_plus_30,ffpd_plus_30,sxpd_plus_30,everDPD_3,everDPD_15,everDPD_30
0,6b3e7079-6cc7-4103-8bb1-2c5c9455e888,34ef815c-eba6-4acb-a6f0-a5c9ece158e6,2023-06-05,2023-06-05,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,06296573-9834-4351-b450-c59f44e9741e,c4cec416-702f-4067-ba46-84088fbc0a5f,2023-06-05,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,c74679c1-92d2-43ac-8b73-3e3ed8b8000a,136032b3-d2f5-43ab-a9d2-33230517857f,2023-06-05,2023-06-05,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,028fa7c5-b2f8-45a2-a738-b670de867b94,32947d50-65c5-448a-9bf1-867ab1a3e52f,2023-06-05,2023-06-05,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,6e7c058d-8863-4c6c-8547-b3bfa0a022ef,198af8d7-fcf0-4636-800b-2b49cdbfbb2d,2023-06-05,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [17]:
querySno = SQLQuery('snowflake')
fico = querySno(r""" 
select a.lending_business_id, 
case when json_extract_path_text(predict_meta,'business_id') is null
     then json_extract_path_text(json_extract_path_text(predict_meta,'data') , 'business_id')
     else json_extract_path_text(predict_meta,'business_id') end as business_id,
b.fico_score
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS a
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."CUSTOMER_CREDIT_PROFILES" b
on a.LENDING_BUSINESS_ID = b.LENDING_BUSINESS_ID
and fico_score is not null
and fico_score != ''
and date(b.created_at) <= date(a.created_at)
where json_extract_path_text(a.rejection_reasons, 'data') = '[]'
or a.is_overridden = 'TRUE'
""")

In [18]:
fico.shape

(5783, 3)

In [19]:
target = base[['lending_business_id','business_id','everDPD_15']]

In [20]:
querySno = SQLQuery('snowflake')
lending_base = querySno(r""" 
with approved as (
select distinct a.lending_business_id, 
case when json_extract_path_text(predict_meta,'business_id') is null
     then json_extract_path_text(json_extract_path_text(predict_meta,'data') , 'business_id')
     else json_extract_path_text(predict_meta,'business_id') end as business_id,
a.created_at,
case when b.lending_business_id is not null then 1 else 0 end as drawn_flag
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS a
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES" b 
on a.lending_business_id = b.lending_business_id
where json_extract_path_text(rejection_reasons, 'data') = '[]'
or is_overridden = 'TRUE'
)

-- select count(distinct business_id) from (
select a.*
from approved a 
join (select distinct business_id from prod_db.adhoc.lending_response_base_w_plaid_match_v2_20230623) b 
on a.business_id = b.business_id 
and a.created_at >= date('2022-11-29') and a.created_at <= date('2023-06-05')
-- )
""")

In [21]:
lending_base.shape

(888, 4)

In [22]:
lending_base.head()

,lending_business_id,business_id,created_at,drawn_flag
0,5249377a-500b-42a7-af3e-109b648c928d,b14a4191-4139-446d-aa66-b071e4eb8ccb,2023-03-24 17:29:54.080531+00:00,1
1,fe14006a-6546-43a3-a1a6-88c4993f8f3f,416d0695-5f0f-4d37-88d8-53ecad0b1f5f,2023-04-25 23:30:57.040311+00:00,1
2,a30ca216-71ee-4822-b543-d9e09532ad99,8a67470b-ba91-498a-b0b0-522902a6f714,2023-01-20 00:29:07.578576+00:00,1
3,24bb963f-d3a9-4159-b893-5912efe3e9a4,dd7f8b41-4ff5-4011-88ea-e244561bd9d3,2022-11-30 17:22:03.691987+00:00,0
4,50966993-54ee-446b-bbcf-c957c82354f2,da9818c4-8f95-417c-ab19-f60669832897,2023-04-20 21:56:17.863761+00:00,0


In [23]:
df1 = lending_base.merge(target,how='left',left_on=['lending_business_id','business_id'],right_on=['lending_business_id','business_id'])
df1.shape

(888, 5)

In [24]:
df1.head()

,lending_business_id,business_id,created_at,drawn_flag,everDPD_15
0,5249377a-500b-42a7-af3e-109b648c928d,b14a4191-4139-446d-aa66-b071e4eb8ccb,2023-03-24 17:29:54.080531+00:00,1,NaN
1,fe14006a-6546-43a3-a1a6-88c4993f8f3f,416d0695-5f0f-4d37-88d8-53ecad0b1f5f,2023-04-25 23:30:57.040311+00:00,1,0.0
2,a30ca216-71ee-4822-b543-d9e09532ad99,8a67470b-ba91-498a-b0b0-522902a6f714,2023-01-20 00:29:07.578576+00:00,1,0.0
3,24bb963f-d3a9-4159-b893-5912efe3e9a4,dd7f8b41-4ff5-4011-88ea-e244561bd9d3,2022-11-30 17:22:03.691987+00:00,0,NaN
4,50966993-54ee-446b-bbcf-c957c82354f2,da9818c4-8f95-417c-ab19-f60669832897,2023-04-20 21:56:17.863761+00:00,0,NaN


In [25]:
# drawn but not aged
df1[(df1['drawn_flag']==1) & (df1['everDPD_15'].isnull())].shape

(95, 5)

In [26]:
df = df1.merge(fico,left_on=['lending_business_id','business_id'],right_on=['lending_business_id','business_id'])
df.shape

(888, 6)

In [27]:
df['dpd_flag'] = 0
df.loc[~(df['everDPD_15'].isnull()),'dpd_flag'] = 1

In [28]:
df['dpd_flag'].sum()

483

In [29]:
df['fico_score'] = df['fico_score'].astype('int')

In [30]:
df['fico_flag'] = 0
df.loc[((df['fico_score']>600) & (df['fico_score']<=630)) | ((df['fico_score']>=720) & (df['fico_score']<=850)),'fico_flag'] = 1

In [32]:
print(df.shape)
print(df['drawn_flag'].sum())
print(df['dpd_flag'].sum())
print(df['everDPD_15'].sum())

(888, 8)
578
483
71.0


In [34]:
df_final = df[(df['dpd_flag']==1) | ((df['dpd_flag']==0) & (df['fico_flag']==1))]
df_final.shape

(708, 8)

In [35]:
df_final['target'] = 0
df_final.loc[df['dpd_flag']==1, 'target'] = df_final['everDPD_15']
df_final.loc[(df['fico_flag']==1) & (df['fico_score']<630), 'target'] = 1

/var/folders/v3/dy2p0xqd3mz5bvjtls49l7t80000gn/T/ipykernel_11873/1725682920.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['target'] = 0


In [41]:
df_final.sample(10)

,lending_business_id,business_id,created_at,drawn_flag,everDPD_15,fico_score,target
873,8cf26f6c-7a8d-4951-aed6-7cf229ce1cbe,6c40cce7-434d-4f06-8823-dde976f76452,2023-01-26 20:02:12.811738+00:00,1,1.0,678,1
667,6274f443-4d48-4632-b2b4-e58b89a0834a,bcc783a4-1a0c-4721-a442-e5178793ff1d,2022-11-30 18:56:24.616727+00:00,1,0.0,710,0
559,0ce6fb49-fd77-4cbb-a7b1-ea941581c3f4,a99e8bf4-2ebc-498f-87d0-c0f7269bc57b,2023-01-13 20:28:30.627066+00:00,1,0.0,683,0
217,1a5874a8-5ad9-4340-aa50-6f63a3dec957,59a345a5-ca8d-4998-81e2-ae85a74fba93,2023-02-23 12:36:36.746526+00:00,1,0.0,638,0
543,b854768e-1903-41e9-ad66-aae065da2773,e62ca860-e7a0-4779-8815-3bfc7e45bc47,2023-05-16 12:45:34.262026+00:00,1,NaN,749,0
860,f75cf355-84a2-48cc-b553-3fffff1e5fa7,32316aeb-3981-47bf-8a85-4d9d22365119,2022-12-06 16:43:14.693514+00:00,1,0.0,608,1
582,a97237da-ec1d-47c5-a1d0-31a485f903aa,69b1a76e-be39-44f5-bcde-fb710b6f1556,2023-03-09 11:25:09.557277+00:00,1,1.0,621,1
746,cc3e99c0-3afd-4a22-93b3-52f4c6a32cc1,f17a0a0f-bb50-4621-be8d-6e5bf6498baa,2023-03-16 14:41:25.773311+00:00,0,NaN,787,0
557,b37efc29-85e6-4e83-a214-35d14af3ad56,08271739-f0d2-4849-ac41-ebb1743ab013,2023-03-08 00:29:04.337316+00:00,0,NaN,850,0
361,7e430af9-c856-437b-b988-dd67dc0a0488,fdd42381-93e9-45aa-a894-54b77a88e968,2023-06-03 16:29:44.381574+00:00,0,NaN,610,1


In [39]:
print('df length = {}'.format(len(df_final)))
print('target rate = {}'.format(df_final['target'].sum()/len(df_final)))

df length = 708
target rate = 0.2033898305084746


In [40]:
df_final = df_final.drop(['dpd_flag','fico_flag'],axis=1)

In [43]:
# from sqlalchemy.types import NVARCHAR
# q = SQLQuery('snowflake')

# df_final.to_sql(name='lending_base_target_23jun',
#                  con=q.engine, 
#                  schema='prod_db.adhoc',
#                  if_exists='append', 
#                  index=False, 
#                  chunksize=16000, 
#                  method='multi',
#                  dtype={col_name: NVARCHAR for col_name in df_final})